# Sentiment Analysis in the Browser

In this notebook, we will show how to create a `.air` file to perform sentiment analysis in the browser using a neural network.  To do this, we will utilize the IMDB Movie Reviews dataset to build the initial model, prune the model using the `mann` package, and then package the model using the `aisquared` Python SDK.

## Dependencies

For this notebook, the following dependencies are required:

- `mann`
- `aisquared`

Both of these are available on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install these dependencies as well as imports them into the notebook environment, along with TensorFlow (which is a dependency of the `mann` package).

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import aisquared

## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first download and preprocess the data, create the model, prune the model so that it can perform well in the browser, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [ ]:
# Loading the data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words = 10000,
    skip_top = 0,
    start_char = 1,
    oov_char = 2,
    index_from = 3
)
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    x_test,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)

# Get the vocabulary
vocab = tf.keras.datasets.imdb.get_word_index()

# Add 2 to each vocab value to ensure matching with the needed values
vocab = {
    k : v + 2 for k, v in vocab.items()
}


In [ ]:
# Create the model

input_layer = tf.keras.layers.Input(x_train.shape[1:])
embedding_layer = tf.keras.layers.Embedding(
    10000,
    4
)(input_layer)
x = tf.keras.layers.Flatten()(embedding_layer)
for _ in range(5):
    x = tf.keras.layers.Dense(1000, activation = 'relu')(x)
output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
# Prune the model and train it
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model with the sparsification callback
model.fit(
    x_train,
    y_train.reshape(-1,1),
    epochs = 20,
    batch_size = 512,
    validation_split = 0.2,
    verbose = 2,
    callbacks = None
)

In [ ]:
# Check model performance
preds = (model.predict(x_test) >= 0.5).astype(int)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('SentimentClassifier.h5')

## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package `DocumentPredictor` class.

In [ ]:
harvester = aisquared.config.harvesting.TextHarvester()

preprocessor = aisquared.config.preprocessing.TextPreprocessor(
    [
        aisquared.config.preprocessing.Tokenize(),
        aisquared.config.preprocessing.ConvertToCase(),
        aisquared.config.preprocessing.RemoveCharacters(),
        aisquared.config.preprocessing.ConvertToVocabulary(vocabulary = vocab, max_vocab = 9999),
        aisquared.config.preprocessing.PadSequences(length = 512, pad_location = 'post', truncate_location = 'post')
    ]
)   
analytic = aisquared.config.analytic.LocalModel('SentimentClassifier.h5', 'text')

##we might be missing an analogue to "sequence_length" in the .config package

postprocessor = aisquared.config.postprocessing.BinaryClassification(['positive', 'negative'], 0.5)

renderer = aisquared.config.rendering.DocumentRendering(include_probability = True)

feedback = aisquared.config.feedback.BinaryFeedback(['positive', 'negative'])

aisquared.config.ModelConfiguration(
    'SentimentClassifier',
    harvester,
    preprocessor,
    analytic,
    postprocessor,
    renderer,
    feedback).compile(dtype = 'float16')